# Index data on file system into easily navigable database

In [1]:
import json
import numpy as np
import os

from data import DataLoader
from db import d_models
from db import make_session, empty_tables

import CONFIG as C
import LOCAL_SETTINGS as L
import PARAMS as P

In [2]:
def write_trials_from_file_system():
    """
    Look in the file system and extract paths, etc., of all the data files and store them
    in the database.
    """
    session = make_session()
    empty_tables(session, d_models.Trial)

    # get directories for different experiments

    for expt_dir in next(os.walk(L.DATA_ROOT))[1]:
        print('LOADING DATA FILES FROM EXPERIMENT: "{}".'.format(expt_dir))

        # get all of the bottom level directories (which contain the data files)
        data_dirs = []

        for root, dirs, files in os.walk(os.path.join(L.DATA_ROOT, expt_dir)):
            if not dirs: data_dirs.append(root)

        # store trial for each data directory in database
        data_ctr = 0

        for data_dir in data_dirs:

            print('TRIAL: "{}"'.format(os.path.basename(data_dir)))

            trial = d_models.Trial()

            trial.name = os.path.basename(data_dir)
            trial.expt = expt_dir
            trial.fly = '.'.join(trial.name.split('.')[:2])
            trial.number = int(trial.name.split('.')[2])

            trial.path = os.path.relpath(data_dir, L.DATA_ROOT)

            # get names of all files in trial directory
            file_names = next(os.walk(data_dir))[2]

            try:
                file_name_behav = [
                    fn for fn in file_names if fn.endswith(C.BEHAV_FILE_ENDING)][0]
            except:
                raise Exception(
                    'Could not find ".dat" file in directory: "{}"'.format(
                        data_dir))

            if True:
                try:
                    file_name_gcamp = [
                        fn for fn in file_names if fn == C.ROI_PROFILES_FILE][0]
                except:
                    raise Exception(
                        ('Could not find "ROI-profiles" file in directory: '
                         '"{}"'.format(data_dir)))

                try:
                    file_name_gcamp_timestamp = [
                        fn for fn in file_names if fn == C.GCAMP_FILE
                    ][0]
                except:
                    raise Exception(
                        ('Could not find GCaMP timestamp file in directory: '
                         '"{}"'.format(data_dir)))

                try:
                    file_name_light_times = [
                        fn for fn in file_names if fn == C.LIGHT_TIMES_FILE
                    ][0]
                except:
                    raise Exception(
                        ('Could not find light times file in directory: '
                         '"{}"'.format(data_dir)))

            else:
                try:
                    file_name_gcamp = [
                        fn for fn in file_names
                        if fn.endswith('.csv') and 'Auto' not in fn][0]
                except:
                    raise Exception(
                        'Could not find ".csv" file in directory: "{}"'.format(
                            data_dir))

                file_name_gcamp_timestamp = None
                file_name_light_times = None

            # see if there is a file defining air tube motion
            file_name_air_tube = C.AIR_TUBE_FILE \
                if C.AIR_TUBE_FILE in file_names else None

            # see if there is a details file
            if 'details.json' in file_names:
                path = os.path.join(data_dir, 'details.json')
                with open(path) as f: details = json.load(f)
            else:
                details = {}

            trial.file_name_behav = file_name_behav
            trial.file_name_gcamp = file_name_gcamp
            trial.file_name_gcamp_timestamp = file_name_gcamp_timestamp
            trial.file_name_light_times = file_name_light_times
            trial.file_name_air_tube = file_name_air_tube
            
            trial.details = details

            session.add(trial)

            data_ctr += 1

        session.commit()

        print(('{}/{} TRIALS LOADED FOR EXPERIMENT: '
               '"{}".'.format(data_ctr, len(data_dirs), expt_dir)))

    print('Complete.')
    
write_trials_from_file_system()

LOADING DATA FILES FROM EXPERIMENT: "closed_loop".
TRIAL: "20160904.Fly4.2"
TRIAL: "20160904.Fly4.3"
TRIAL: "20160904.Fly4.6"
TRIAL: "20160904.Fly4.7"
TRIAL: "20160904.Fly5.3"
TRIAL: "20160904.Fly5.4"
TRIAL: "20160904.Fly6.1"
TRIAL: "20160904.Fly6.2"
TRIAL: "20160904.Fly6.3"
TRIAL: "20160904.Fly6.4"
TRIAL: "20160904.Fly6.7"
TRIAL: "20160904.Fly6.8"
TRIAL: "20160905.Fly1.1"
TRIAL: "20160905.Fly1.2"
TRIAL: "20160905.Fly1.3"
TRIAL: "20160905.Fly1.4"
TRIAL: "20160905.Fly1.5"
TRIAL: "20160905.Fly1.6"
TRIAL: "20160905.Fly1.7"
TRIAL: "20160905.Fly2.1"
TRIAL: "20160905.Fly2.2"
TRIAL: "20160905.Fly2.3"
TRIAL: "20160905.Fly2.4"
TRIAL: "20160905.Fly4.1"
TRIAL: "20160905.Fly4.2"
TRIAL: "20160905.Fly4.3"
TRIAL: "20160905.Fly4.4"
TRIAL: "20160905.Fly4.5"
TRIAL: "20160905.Fly4.6"
TRIAL: "20160905.Fly4.7"
TRIAL: "20160905.Fly5.1"
TRIAL: "20160905.Fly5.2"
TRIAL: "20160905.Fly5.4"
TRIAL: "20160905.Fly6.1"
TRIAL: "20160905.Fly6.2"
TRIAL: "20160905.Fly6.3"
TRIAL: "20160905.Fly6.4"
TRIAL: "20160905.Fly7.1"

# Upload hand-identified walking-speed thresholds to database

These are used to label each timepoint in a trial as belonging to either a "paused", "walking", or "ambiguous" state. They were determined by hand on a trial-by-trial basis and are stored in PARAMS.py.

In [6]:
def write_speed_thresholds_from_params():
    session = make_session()
    
    for name, th in P.SPEED_THRESHOLDS.items():
        
        trial = session.query(d_models.Trial).filter_by(name=name).first()
        trial.walking_threshold = th
        
        session.add(trial)
        session.commit()
        
    for trial in session.query(d_models.Trial).all():
        
        if trial.walking_threshold:
            print('Walking speed threshold for trial "{}" ='.format(trial.name))
            print(trial.walking_threshold)
        
    session.close()
        
write_speed_thresholds_from_params()

Walking speed threshold for trial "20160904.Fly4.2" =
None
Walking speed threshold for trial "20160904.Fly4.3" =
None
Walking speed threshold for trial "20160904.Fly4.6" =
None
Walking speed threshold for trial "20160904.Fly4.7" =
None
Walking speed threshold for trial "20160904.Fly5.3" =
None
Walking speed threshold for trial "20160904.Fly5.4" =
None
Walking speed threshold for trial "20160904.Fly6.1" =
None
Walking speed threshold for trial "20160904.Fly6.2" =
None
Walking speed threshold for trial "20160904.Fly6.3" =
None
Walking speed threshold for trial "20160904.Fly6.4" =
None
Walking speed threshold for trial "20160904.Fly6.7" =
None
Walking speed threshold for trial "20160904.Fly6.8" =
None
Walking speed threshold for trial "20160905.Fly1.1" =
None
Walking speed threshold for trial "20160905.Fly1.2" =
None
Walking speed threshold for trial "20160905.Fly1.3" =
None
Walking speed threshold for trial "20160905.Fly1.4" =
None
Walking speed threshold for trial "20160905.Fly1.5" =
No